# Ford raw time series EDA
The goals of this notebook are to:
- visualize and get an intuition of how the ford data is like.  
- Find a lead to estimate the soh.

## Imports

In [ ]:
from datetime import datetime as DT
import pytz

import pandas as pd
from pandas import DataFrame as DF
import plotly.express as px

from core.s3_utils import S3_Bucket
from core import *
from core.pandas_utils import *
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries



## Setup

In [ ]:
tss = ProcessedTimeSeries("ford")
tss.columns

## Time series processing
We need to display some first graph to get an overview of the available data.

In [ ]:
tss.count() / len(tss)

In [ ]:
ts = raw_tss.xs("WF0TK1EM2MMA16501", level=0)
COLS_TO_DISPLAY = [
    'battery_energy',
    'soc',
    #'capacity',
]
twinx(ts, ['battery_energy'], ["soc"], x_col="date")

The battery energy is a direct function of the battery level.
This is a first track to calculate the SoH.

In [ ]:
# Instead of ts["capacity"].type()
print("Data type of 'capacity':", ts["capacity"].dtype)
print("Unique values in 'capacity':", ts["capacity"].unique())
print("Value counts of 'capacity':\n", ts["capacity"].value_counts())



In [ ]:
ts['capacity'] = pd.to_numeric(ts['capacity'], errors='coerce')

ts["SoH"] = ts["battery_energy"] / ts["soc"] / ts["capacity"] * 100
px.scatter(ts, x="soc", y=["SoH"]).show()

Depending on the battery level, the SoH is between 0.85 and 0.95.
We can add a filter to have a better understanding of the SoH.

# Adding Filter 

In [ ]:
tss_filtered = ts.query("soc > 0.5")
px.scatter(tss_filtered, x="soc", y=["SoH"]).show()

# Adding filter on the charge/discharge